Having created the updated, more accurate solar raster files, I need to create cutouts for the deepnet.

I will create one massive polygon from all the tiles and another with all the panels. I will then randomly sample points within the polygons, and create square cutouts centered at those points.

## Make polygon of all Exeter tiles

In [169]:
import rasterio
from rasterio.plot import show
import PIL
import glob
import pylab as plt
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import measure
import geopandas as gpd
from fiona.crs import from_epsg
from shapely.geometry import box


In [170]:
training_path='/Volumes/pdh_storage_2/OCF/PV_Mapping/data/raw/Exeter/'
aerial_files=glob.glob(training_path+'GeoTiff_*/*.tif')
solar_files=glob.glob(training_path+'solar-rasters_v2/*.tif')

In [171]:
dataset = rasterio.open(solar_files[0])
dataset.bounds

BoundingBox(left=266000.0, bottom=105000.0, right=267000.0, top=106000.0)

In [208]:
geo=[]
for i,d in enumerate(solar_files):
    dataset = rasterio.open(d)
    geo.append(gpd.GeoDataFrame({"id":1,"geometry":[box(*dataset.bounds)],'tile':d.split('/')[-1].split('.tif')[0]}))
polygons_all=pd.concat(geo)
polygons_all.reset_index(inplace=True)
polygons_all=polygons_all.set_crs(epsg=27700)
#save as geojson
polygons_all.to_file(filename='Exeter_tiles_orig.geojson', driver='GeoJSON')

Process for making cutouts
1. Use the geopandas dissolve (as documented [here](https://stackoverflow.com/questions/34325030/merging-two-geojson-polygons-in-python)) to create union of tiles
2. Use pointpats to create random points within polygon
Use GeoSeries.buffer to create buffer around panels polygon
3. make cutouts around those points, checking that there are no solar panels in the negative images and that the box isnt hitting tile boundary

Combine the polygons for all the tiles to get a mask from which to generate random points

In [210]:
polygons_all['new_column'] = 0
polygons_new = polygons_all.dissolve(by='new_column')

In [212]:
polygons_new.to_file(filename='Exeter_tiles_union.geojson', driver='GeoJSON')

Use [`pointpats`](https://github.com/pysal/pointpats/blob/master/notebooks/process.ipynb) to generate random points within the maskpolygon

In [233]:
from pointpats import PoissonPointProcess, PoissonClusterPointProcess, Window, poly_from_bbox, PointPattern
import libpysal

In [60]:
window=Window(libpysal.cg.asShape(polygons_new.geometry[0]).parts)

In [65]:
np.random.seed(5)
#note as conditioning=False and asPP=False makes this a simulate a N-conditioned CSR and point series respectively
samples = PoissonPointProcess(window, 200, 1, conditioning=False, asPP=False)

Creating samples works, now I need to create mask around the panels, including a buffer and combine so that I have the following masks from which to generate samples:
1. A non-panel mask = aerial tiles union - buffer panel mask (- aerial tiles buffer)
2. A panel mask = panel mask union (- aerial tiles buffer)

To make it simpler when making cutouts, I will be subtracting the aerial tiles buffer. This a buffer around the edge of the tiles so that I can generate square cutouts without having to worry about going over numerous tif files. I could make the cutouts such that if one did cross a tiff boundary it could deal with it.

In [213]:
geopolygons=glob.glob(training_path+'/polygons/*.geojson')

In [218]:
panel_gpd=[]
for i,p in enumerate(geopolygons):
    panel_gpd.append(gpd.read_file(p).set_crs(epsg=27700,allow_override=True))
all_panels_gpd=pd.concat(panel_gpd)
#reindex the dataframe
all_panels_gpd.reset_index(inplace=True)


In [220]:
#set buffer size
buffer_size=10

In [221]:
all_panels_gpd_buffer=all_panels_gpd.buffer(buffer_size)
all_panels_gpd_buffer.to_file(filename='all_panels_buffer.geojson', driver='GeoJSON')

In [222]:
all_panels_gpd.to_file(filename='all_panels.geojson', driver='GeoJSON')

#### create aerial tile buffer

In [223]:
aerial_tile_buffer=polygons_all.buffer(buffer_size)


In [224]:
aerial_tile_buffer.to_file(filename='buffered_tiles.geojson', driver='GeoJSON')

In [225]:
aerial_tile_buffer_df=gpd.GeoDataFrame(geometry=aerial_tile_buffer)

In [226]:
buffered_tiles=[]
for i in range(0,len(aerial_tile_buffer_df)):
    buffered_tiles.append(gpd.overlay(aerial_tile_buffer_df.iloc[[i]],aerial_tile_buffer_df.drop(i), how='difference'))

In [227]:
buffered_tiles=pd.concat(buffered_tiles)
buffered_tiles.to_file(filename='buffered_tiles_intersect.geojson', driver='GeoJSON')

Make final masks



In [229]:
# neen to create a small buffer for intersction to work (as described here https://www.programmersought.com/article/69515213493/)
all_panels_gpd_buffer_small=all_panels_gpd.buffer(0.1)


In [439]:
#the first overlay subtracts the buffered panels, the outer overlay deals with the outer buffered tiles that extend beyond original tile
negative_mask=gpd.overlay(gpd.overlay(buffered_tiles,gpd.GeoDataFrame(geometry=all_panels_gpd_buffer),how='difference'),polygons_new,how='intersection')

positive_mask=gpd.overlay(buffered_tiles,gpd.GeoDataFrame(geometry=all_panels_gpd_buffer_small),how='intersection')

In [440]:
negative_mask.to_file(filename='negative_mask.geojson', driver='GeoJSON')
positive_mask.to_file(filename='positive_mask.geojson', driver='GeoJSON')

## Generate positions of cutouts

In [441]:
negative_mask['new_column'] = 0
negative_mask = negative_mask.dissolve(by='new_column')
#positive_mask['new_column'] = 0
#positive_mask = positive_mask.dissolve(by='new_column')

In [442]:
window=Window(libpysal.cg.asShape(negative_mask.geometry[0]).parts)
np.random.seed(5)
#note as conditioning=False and asPP=False makes this a simulate a N-conditioned CSR and point series respectively
negative_samples = PoissonPointProcess(window, 2000, 1, conditioning=False, asPP=False)

In [443]:
neg_geo=gpd.GeoDataFrame(geometry=gpd.points_from_xy(negative_samples.realizations[0][:,0],negative_samples.realizations[0][:,1]))
neg_geo=neg_geo.set_crs(epsg=27700)
neg_geo.to_file(filename='neg_samples.geojson', driver='GeoJSON')

The positive point process takes too long because it is not optimized to deal with multipolygons. One way to do this would be to sample from discrete distribution, weighted by fractional area of each polygon

NOTE: At the moment, the solar farms are going to be picked more, I may want to do something to lessen the weight on them.

In [421]:
positive_mask['weight']=positive_mask.area/positive_mask.area.sum()

In [422]:
npos=2000
positive_samples=[]
pos_polygon=np.random.choice(positive_mask.index,npos, p=positive_mask['weight'])
for i in range(0,npos):
    window=Window(libpysal.cg.asShape(positive_mask.geometry[pos_polygon[i]]).parts)
    positive_samples.append(PoissonPointProcess(window, 1, 1, conditioning=False, asPP=False).realizations[0])

In [423]:
positive_samples=np.asarray(positive_samples).reshape(npos,2)

In [424]:
pos_geo=gpd.GeoDataFrame(geometry=gpd.points_from_xy(positive_samples[:,0],positive_samples[:,1]))
pos_geo=pos_geo.set_crs(epsg=27700)
pos_geo.to_file(filename='pos_samples.geojson', driver='GeoJSON')

To make cutouts, I need to:
    1. find what tile each of the points are in
    2. make cutout, centered on that position for
        1. Aerial image
        2. binary image

In [444]:
#get the tile for each positive and negative sample
pos_geo['tile']=''
neg_geo['tile']=''
for i in range(len(polygons_all)):
    pos_geo.loc[pos_geo.within(polygons_all.loc[i, 'geometry']).values,'tile']=polygons_all.loc[i, 'tile']
    neg_geo.loc[neg_geo.within(polygons_all.loc[i, 'geometry']).values,'tile']=polygons_all.loc[i, 'tile']

In [426]:
from shapely.geometry import box, mapping
import fiona
from rasterio import transform
from fiona.crs import from_epsg
import pycrs
from rasterio.mask import mask

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

def make_aerial_cutout(tif_file,centre_point,width,out_tif):
    dataset = rasterio.open(tif_file)
    
    bbox = box(centre_point.xy[0][0]-width,centre_point.xy[1][0]-width,centre_point.xy[0][0]+width,centre_point.xy[1][0]+width)
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(27700))
    geo = geo.to_crs(crs=dataset.crs.data)
    coords = getFeatures(geo)
    out_img, out_transform = mask(dataset, shapes=coords, crop=True)
    #crop solar raster file
    #rast=np.empty_like(out_img[0,:,:]).astype(float)
    #rast[:,:]=0.8
    #rast[ymin_np:ymax_np,xmin_np:xmax_np]=1.2
    #out_img=out_img*rast
    # Copy the metadata
    out_meta = dataset.meta.copy()

    # Parse EPSG code
    epsg_code = int(dataset.crs.data['init'][5:])

    out_meta.update({"driver": "GTiff",
         "height": out_img.shape[1],
         "width": out_img.shape[2],
         "transform": out_transform})
    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_img.astype(np.uint8))
        
def make_binary_cutout(tif_file,centre_point,width,out_tif):
    dataset = rasterio.open(tif_file)
    bbox = box(centre_point.xy[0][0]-width,centre_point.xy[1][0]-width,centre_point.xy[0][0]+width,centre_point.xy[1][0]+width)
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(27700))
    geo = geo.to_crs(crs=dataset.crs.data)
    coords = getFeatures(geo)
    out_image, out_transform = mask(dataset, shapes=coords, crop=True)

    out_meta = dataset.meta.copy()

    # Parse EPSG code
    epsg_code = int(dataset.crs.data['init'][5:])


    
    out_meta.update({"driver": "GTiff",
     "height": out_image.shape[1],
     "width": out_image.shape[2],
     "transform": out_transform,
     "count":1,
     'nodata':0.0,
     'compress':'lzw'})
    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_image.astype(rasterio.uint8))

In [429]:
#loop over every sample and make both cutouts for both aerial and binary tif
for i in range(0,len(pos_geo)):
    out_path='/Volumes/pdh_storage_2/OCF/PV_Mapping/data/raw/Exeter/deepnet_cutouts/positive/'
    aerial_file=list(filter(lambda x: pos_geo['tile'][i].split('_')[0] in x, aerial_files))[0]
    solar_file=list(filter(lambda x: pos_geo['tile'][i] in x, solar_files))[0]
    make_aerial_cutout(aerial_file,pos_geo['geometry'][i],10,out_path+'train_{}.tif'.format(i))
    make_binary_cutout(solar_file,pos_geo['geometry'][i],10,out_path+'train_binary_{}.tif'.format(i))

/Users/pdh21/anaconda3/envs/m5_forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [445]:
#loop over every sample and make both cutouts for both aerial and binary tif
for i in range(0,len(neg_geo)):
    out_path='/Volumes/pdh_storage_2/OCF/PV_Mapping/data/raw/Exeter/deepnet_cutouts/negative/'
    aerial_file=list(filter(lambda x: neg_geo['tile'][i].split('_')[0] in x, aerial_files))[0]
    solar_file=list(filter(lambda x: neg_geo['tile'][i] in x, solar_files))[0]
    try:
        make_aerial_cutout(aerial_file,neg_geo['geometry'][i],10,out_path+'train_{}.tif'.format(i))
        make_binary_cutout(solar_file,neg_geo['geometry'][i],10,out_path+'train_binary_{}.tif'.format(i))
    except (ValueError) as e:
        print(i,neg_geo['tile'][i],aerial_file,solar_file)

/Users/pdh21/anaconda3/envs/m5_forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
